In [99]:
%%javascript

require(['widgets/js/widget', 'widgets/js/manager', 'jquery', 'underscore'], function(widget, manager, $, _) {
    var GoogleMapsView = widget.DOMWidgetView.extend({
        initialize: function() {
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                }
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'], 
                    function() {}, 
                    function() {});
            }
        },
        
        render: function() {
            $.when(this.mapsDeferred.promise()).then(_.bind(function() {
                this.$el.empty();
                _.defer(_.bind(function() {
                    this.$map = $(
                        '<div style="height: ' + this.model.get('height') + '; width: ' + this.model.get('width') + ';"></div>');
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {lat: this.model.get('lat'), lng: this.model.get('lng')},
                        zoom: this.model.get('zoom')
                    });
                    
                    this.listenTo(this.model, 'change:height', _.bind(function() {
                        this.$map.height(this.model.get('height'));
                        google.maps.event.trigger(this.map, 'resize');
                    }, this));
                    this.listenTo(this.model, 'change:width', _.bind(function() {
                        this.$map.width(this.model.get('width'));
                        google.maps.event.trigger(this.map, 'resize');
                    }, this));
                    
                    google.maps.event.addListener(this.map, 'bounds_changed', _.bind(this.syncFromMap, this));
                    
                    this.listenTo(this.model, 'change:lat', _.bind(this.syncFromModel, this));
                    this.listenTo(this.model, 'change:lng', _.bind(this.syncFromModel, this));
                    this.listenTo(this.model, 'change:zoom', _.bind(this.syncFromModel, this));
                    
                    this.listenTo(this.model, 'msg:custom', _.bind(this.handleMessage, this));
                }, this));
            }, this));
        },
        
        syncFromMap: function() {
            this.model.set({
                lat: this.map.getCenter().lat(),
                lng: this.map.getCenter().lng(),
                zoom: this.map.getZoom()
            });
            this.model.save_changes();
        },
        
        syncFromModel: function() {
            this.map.setCenter(new google.maps.LatLng(
                this.model.get('lat'),
                this.model.get('lng')
            ));
            this.map.setZoom(this.model.get('zoom'));
        },
        
        handleMessage: function(payload) {
            var params = payload.params || [];
            this[payload.method].apply(this, params);
        },
        
        addLayer: function() {
            console.log(arguments);
        }
    });
    
    manager.WidgetManager.register_widget_view('GoogleMapsView', GoogleMapsView);
});

<IPython.core.display.Javascript object>

In [100]:
from ipywidgets import widgets
import traitlets

class GoogleMapsView(widgets.DOMWidget):
    _view_name = traitlets.Unicode('GoogleMapsView', sync=True)
    lat = traitlets.CFloat(0, sync=True)
    lng = traitlets.CFloat(0, sync=True)
    zoom = traitlets.CInt(2, sync=True)
    
    height = traitlets.CUnicode('300px', sync=True)
    width = traitlets.CUnicode('400px', sync=True)
    
    def addLayer(self, image, vis_params):
        # map_id = image.getMapId(vis_params)
        self.send({'method': 'addLayer', 'params': [1, 2, 3, 4]})

In [101]:
from IPython.display import display

import ee

a = GoogleMapsView()
b = GoogleMapsView()

widgets.jslink((a, 'lat'), (b, 'lat'))
widgets.jslink((a, 'lng'), (b, 'lng'))
widgets.jslink((a, 'zoom'), (b, 'zoom'))

In [102]:
a

In [103]:
b

In [104]:
a.lat

0.0

In [105]:
b.lat

0.0

In [106]:
a.addLayer(None, None)

In [107]:
a.width = '600px'